In [2]:
import sys
sys.path.append('..')
from MPRA_exp.utils import *

os.chdir('data')

In [14]:
def save_to_fasta(sequences, filename):
    """
    将一组DNA序列保存为FASTA格式文件。
    
    :param sequences: 一个包含seq的列表
    :param filename: 保存的文件名，例如 "sequences.fasta"
    """
    with open(filename, 'w') as file:
        for seq_id, sequence in enumerate(sequences):
            file.write(f">{seq_id}\n")
            file.write(f"{sequence}\n")

# df = pd.read_csv('../../data/GosaiMPRA/GosaiMPRA_total.csv')
# save_to_fasta(df['seq'], 'GosaiMPRA_total.fa')

df = pd.read_csv('../../../data/SirajMPRA/SirajMPRA_total.csv')
print(len(df))
save_to_fasta(df['seq'], 'SirajMPRA_total.fa')

585605


In [ ]:
subprocess.run('split -l 200000 SirajMPRA_total.fa seq_part_', shell=True)
subprocess.run('ls seq_part_* > file_list.txt', shell=True)
subprocess.run('cat file_list.txt | parallel -j 30 "fimo --skip-matched-sequence --no-pgc --max-stored-scores 10000000 JASPAR2024_CORE_vertebrates_non-redundant_pfms_meme.txt {} > {}_fimo_output.txt"', shell=True)
subprocess.run('cat seq_part_*_fimo_output.txt > fimo_output.txt', shell=True)

In [4]:
fimo_df = pd.read_csv('Gosai_fimo_output.txt', sep='\t')
# 删除合并时多余的表头
fimo_df = fimo_df[fimo_df['motif_id'] != 'motif_id']

fimo_df

/tmp/ipykernel_1520211/1930870733.py:1: DtypeWarning: Columns (2,3,4,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  fimo_df = pd.read_csv('Gosai_fimo_output.txt', sep='\t')


,motif_id,motif_alt_id,sequence_name,start,stop,strand,score,p-value,q-value,matched_sequence
0,MA0069.1,PAX6,141,93,106,+,13.4,0.000012,NaN,NaN
1,MA0069.1,PAX6,142,93,106,+,16.2778,0.000001,NaN,NaN
2,MA0069.1,PAX6,328,107,120,+,12.7667,0.000018,NaN,NaN
3,MA0069.1,PAX6,329,107,120,+,12.7667,0.000018,NaN,NaN
4,MA0069.1,PAX6,395,59,72,-,11.5667,0.000038,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
40057113,MA0619.2,LIN54,839791,79,85,+,11.5873,0.000097,NaN,NaN
40057114,MA0619.2,LIN54,839809,61,67,-,11.5873,0.000097,NaN,NaN
40057115,MA0619.2,LIN54,839829,97,103,+,11.5873,0.000097,NaN,NaN
40057116,MA0619.2,LIN54,839838,153,159,+,11.5873,0.000097,NaN,NaN


In [21]:
# 读取FIMO输出文件
fimo_df = pd.read_csv('Siraj_fimo_output.txt', sep='\t')

# 删除多余的表头行
fimo_df = fimo_df[fimo_df['motif_id'] != 'motif_id']
fimo_df['sequence_name'] = fimo_df['sequence_name'].astype(int)

# 生成序列和转录因子hits矩阵
counts_df = fimo_df.groupby(['sequence_name', 'motif_alt_id']).size().unstack(fill_value=0)
counts_df

/tmp/ipykernel_1520413/3589478430.py:2: DtypeWarning: Columns (2,3,4,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  fimo_df = pd.read_csv('Siraj_fimo_output.txt', sep='\t')


motif_alt_id,ARGFX,ARNT2,ASCL1,ATF2,ATF3,ATF4,ATF6,ATF7,ATOH7,Alx1,...,ZSCAN4,Zbtb2,Zfp335,Zfp809,Zfp961,Zfx,Zic1::Zic2,Zic2,Zic3,Znf423
sequence_name,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
585601,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
585602,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [23]:
counts_df.columns

Index(['ARGFX', 'ARNT2', 'ASCL1', 'ATF2', 'ATF3', 'ATF4', 'ATF6', 'ATF7',
       'ATOH7', 'Alx1',
       ...
       'ZSCAN4', 'Zbtb2', 'Zfp335', 'Zfp809', 'Zfp961', 'Zfx', 'Zic1::Zic2',
       'Zic2', 'Zic3', 'Znf423'],
      dtype='object', name='motif_alt_id', length=730)

In [24]:
counts_df.index

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       585595, 585596, 585597, 585598, 585599, 585600, 585601, 585602, 585603,
       585604],
      dtype='int64', name='sequence_name', length=585600)

In [26]:
# 创建包含所有索引的完整索引
complete_index = list(range(585605))

# 重新索引 counts_df，以包含所有索引
counts_df = counts_df.reindex(complete_index, fill_value=0)

# 查看结果
counts_df

motif_alt_id,ARGFX,ARNT2,ASCL1,ATF2,ATF3,ATF4,ATF6,ATF7,ATOH7,Alx1,...,ZSCAN4,Zbtb2,Zfp335,Zfp809,Zfp961,Zfx,Zic1::Zic2,Zic2,Zic3,Znf423
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
585601,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
585602,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
585603,0,0,2,0,0,0,0,0,0,0,...,7,0,0,0,0,0,0,0,0,0


In [27]:
TF_names = load_txt('TF_names.txt')

counts_df = counts_df.reindex(columns=TF_names, fill_value=0)

counts_df

motif_alt_id,Mecom,BATF,CTCF,Tcf12,RORC,OSR1,PBX3,VENTX,OTX2,Smad4,...,POU3F1,JDP2,TFAP2C,REST,ZNF331,IRF8,ZNF417,SRY,TWIST1,ASCL1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
585601,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
585602,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
585603,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [28]:
counts_df

motif_alt_id,Mecom,BATF,CTCF,Tcf12,RORC,OSR1,PBX3,VENTX,OTX2,Smad4,...,POU3F1,JDP2,TFAP2C,REST,ZNF331,IRF8,ZNF417,SRY,TWIST1,ASCL1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
585601,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
585602,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
585603,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [29]:
counts_df.to_csv('Siraj_TF_counts_matrix.csv', index=False)

# 后面是纯python方法，很慢

In [ ]:
file_path = "JASPAR2024_CORE_vertebrates_non-redundant_pfms_jaspar.txt"
with open(file_path) as handle:
    motif_list = motifs.parse(handle, "jaspar")

# file_path = "JASPAR2024_CORE_vertebrates_non-redundant_pfms_meme.txt"
# with open(file_path) as handle:
#     motif_list = motifs.parse(handle, "MINIMAL")

print(len(motif_list))
print(len(set(motif_list)))
for motif in motif_list:
    print(motif.name)

In [ ]:
motif_names = list(set([motif.name for motif in motif_list]))
print(len(motif_names))

In [ ]:
save_txt('TF_names.txt', motif_names)

In [ ]:
def to_array(pwm):
    pwm = np.array([pwm[nuc] for nuc in 'ACGT'])
    return pwm

In [ ]:
def scan_pwm(onehot, pwm):
    pwm = pwm.T
    pwm_len = len(pwm)
    scores = np.zeros(len(onehot) - pwm_len + 1)
    for i in range(len(scores)):
        window = onehot[i:i + pwm_len]
        scores[i] = np.sum(window * pwm)
    return scores

In [ ]:
onehot = str2onehot(df.at[1, 'seq'])
pwm = to_array(motif_list[0].pwm)

print(onehot.shape, pwm.shape)
scores = scan_pwm(onehot, pwm)
print(scores.shape)

In [ ]:
score_matrix = np.zeros((len(df), len(motif_list)))
for j, motif in enumerate(motif_list):
    for i, seq in tqdm(enumerate(df['seq']), total=len(df)):
        onehot = str2onehot(seq)
        # pwm = to_array(motif.pwm)
        # scores = scan_pwm(onehot, pwm)
        # score_matrix[i, j] = np.max(scores)

        # rpssm = motif.pssm.reverse_complement()
        results = motif.pssm.search(seq, threshold=0)
        scores = [result[1] for result in results]
        if scores:
            score_matrix[i, j] = np.max(scores)